In [1]:
import pandas as pd 
import time
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import random
import numpy as np
import itertools
import math
from itertools import chain
import operator


In [2]:
def task_reward(members, competency_lists, request):
    # task utility is the sum the competency of coalition members.
    # request: task requested capabilities
    if members == []:
        return 0
    else:
        coal_pro = [competency_lists[mem] for mem in members]
        # return sum([sum([agent_pro[cap] for agent_pro in coal_pro]) for cap in request])
        return sum([max([agent_pro[cap] for agent_pro in coal_pro]) for cap in request])



def sys_reward(CS, competency_lists, t_caps_lists):
    ini_task_u = [[] for j in range(0, len(CS)-1)]
    for j in range(0, len(CS)-1):
        members = CS[j]
        if not members:
            ini_task_u[j] = 0
        else:
            ini_task_u[j] = task_reward(members, competency_lists, t_caps_lists[j])
    global_u = sum(ini_task_u)
    return global_u


def agent_con(competency_lists, t_caps_lists, query_agentIndex, query_taskIndex, member, a_taskInds):
    # the marginal contribution of an agent to a coalition

    if query_taskIndex == len(t_caps_lists):
        return 0
    if query_taskIndex not in a_taskInds[query_agentIndex]:
        return 0
    cur_reward = task_reward(member, competency_lists, t_caps_lists[query_taskIndex])
    if query_agentIndex in member:
        remained_mems = member[:]
        remained_mems.remove(query_agentIndex)  # the remained coalition members besides of agent i
        new_reward = task_reward(remained_mems, competency_lists, t_caps_lists[query_taskIndex])
        return cur_reward - new_reward
    else:
        added_mems = member[:]
        added_mems.append(query_agentIndex)
        new_reward = task_reward(added_mems, competency_lists, t_caps_lists[query_taskIndex])
        return new_reward - cur_reward

In [3]:
# def generate_noise(mean = 0, sd = 0.3): # generate a gussian noise, by defult 10% variation
#     return np.random.normal(mean,sd)


# Functions for generating environment with capability
# generating task
def gen_tasks(task_num, max_capNum, capabilities): # n is the number of task, max_capNum is the maximum number of cap a task could require
    return [sorted(np.random.choice(capabilities, np.random.randint(1, max_capNum+1),replace=False)) for j in range(0, task_num)]



def gen_agents(a_taskInds, tasks, max_capNum, capabilities,
               max_capVal, p_min = 1, p_max=20, sd=0.3):  # m is the number of task, max_capNum is the maximum number of cap a task could require, max_capVal is the maximum capability value

    # tasks: the lists of caps requested by tasks
    agent_num = len(a_taskInds)
    caps_lists = []
    contri_lists = []
    for i in range(0, agent_num):
        t_caps = [tasks[j] for j in a_taskInds[i]]  # lists of caps that each agent could perform

        caps_union = set(itertools.chain(*t_caps))  # union of caps of tasks that agent could perform
        a_cap_num = np.random.randint(min(1, max_capNum, len(caps_union)),
                                      min(len(caps_union), max_capNum) + 1)  # the num of caps the agent will have

        a_caps = set([np.random.choice(t_c) for t_c in
                      t_caps])  # initial draw to guarantee the agent has some contribution to each of the task that he could do

        rest_choices = list(caps_union.difference(a_caps))
        if rest_choices != []:
            update_len = max(0, a_cap_num - len(a_taskInds[i]))
            a_caps.update(np.random.choice(rest_choices, min(len(rest_choices), update_len), replace=False))

        caps_lists.append(sorted(list(a_caps)))

        contri_lists.append(
            [(np.random.randint(1, max_capVal + 1) if c in caps_lists[i] else 0) for c in range(0, len(capabilities))])
    
    # generating agent costs
    a_val = [sum(contri_lists[i]) for i in range(0, agent_num)] # Total cap valus of agents
    a_maxVal = max(a_val) # max total cap value among agents
    a_minVal = min(a_val) # min total cap value among agents

    a_com = [1.0*(a_val[i]-a_minVal)/(a_maxVal-a_minVal) for i in range(0, agent_num)] # min-max scaling to [0,1]
    a_com = [min(1,max(0,a_com[i]+np.random.normal(0,sd))) for i in range(0, agent_num)] # introduce random noise to each agent
    
    a_costs = [int(p_min+a_com[i]*(p_max-p_min))  for i in range(0, agent_num)] # min-max scaling to [p_min, p_max]]

    return caps_lists, contri_lists, a_costs


def random_in_budget(a_tasks, a_costs, task_num, B):
    agent_num = len(a_tasks)
    ava_agents = list(range(0, agent_num))
    alloc = [task_num]*agent_num
    sys_cost = 0
    while True: 
        selec_a = np.random.choice(ava_agents)
        sys_cost += a_costs[selec_a]
        if sys_cost > B:
            break
        else:
            ava_tasks = list(filter(lambda x: x != task_num, a_tasks[selec_a]))
            if ava_tasks==[]:
                ava_tasks = task_num                
            selec_t = np.random.choice(ava_tasks)
            alloc[selec_a] = selec_t
            ava_agents.remove(selec_a)
            if not ava_agents:
                break
    return alloc


# def gen_costs(contri_lists, p_min = 1, p_max=20, mean=0, sd=0.1):
#     # generating agent costs
#     a_val = [sum(contri_lists[i]) for i in range(0, agent_num)] # Total cap valus of agents
#     # print("a_val",a_val)
#     a_maxVal = max(a_val) # max total cap value among agents
#     a_minVal = min(a_val) # min total cap value among agents

#     a_com = [1.0*(a_val[i]-a_minVal)/(a_maxVal-a_minVal) for i in range(0, agent_num)] # min-max scaling to [0,1]
#     a_com = [min(1,max(0,a_com[i]+np.random.normal(mean,sd))) for i in range(0, agent_num)] # introduce random noise to each agent

    
#     a_costs = [int(p_min+a_com[i]*(p_max-p_min))  for i in range(0, agent_num)] # min-max scaling to [p_min, p_max]]
#     return a_costs
    


In [4]:
def cal_MV(cur_con, t_con, assigned = False, cost = 0, algo = 'GNE'):
    if algo == 'GNE':
        return t_con - cur_con  # GreedyNE moveval
    elif assigned: # CF movement values
        return (int(t_con) - int(cur_con))* (10 ** 6)  # the agent has been assigned to a task
    else:
        return 1.0 * (t_con - cur_con) / cost  # the agent has not been assigned to a task
        

def choose_movement(move_vals, agent_set):
    max_vals = [max(move_vals[i]) for i in agent_set]
    v_index, max_v = max(enumerate(max_vals), key=operator.itemgetter(1))
    a_index = agent_set[v_index]
    t_index = move_vals[a_index].index(max_v)
    return a_index, t_index, max_v


def TA_func(t_caps_lists, competency_lists, a_tasks, t_agents, budget, a_costs, cs=[], cur_cost=0, algo='GNE', time_bound=math.inf):  # GNE: GreedyNE; CF:cost efficiency

    start_time = time.time()
    agent_num = len(competency_lists)
    task_num = len(t_caps_lists)

    if cs == []:  # start from scratch
        cs = [[] for j in range(0, task_num + 1)]  # current coalition structure, the last one is dummy coalition
        cs[task_num] = list(range(0, agent_num))
        unassig_agents = list(range(0, agent_num))
        assign_agents = []
        feasible_agents = [i for i in unassig_agents if a_costs[i] <= budget]
        state = [task_num] * agent_num  # each indicate the current task that agent i is allocated to, if = N, means not allocated
        assigned = [False] * agent_num  # indicate whether an agent has been assigned, same as state, but more efficient for some computation
        cur_con = [0] * agent_num
        task_cons = [[agent_con(competency_lists, t_caps_lists, i, j, [], a_tasks) \
                      for j in range(0, task_num + 1)] \
                     for i in range(0, agent_num)]
    else:
        state = [task_num] * agent_num
        assigned = [False] * agent_num
        for j in range(0, task_num):
            for i in cs[j]:
                state[i] = j
                assigned[i] = True
        unassig_agents = cs[task_num][:]
        assign_agents = list(chain.from_iterable([cs[j] for j in range(0, task_num)]))
        feasible_agents_un = [i for i in unassig_agents if a_costs[i] <= budget]
        feasible_agents = assign_agents+feasible_agents_un
        cur_con = [agent_con(competency_lists, t_caps_lists, i, state[i], cs[state[i]], a_tasks) for i in range(0, agent_num)]
        task_cons = [[agent_con(competency_lists, t_caps_lists, i, j, cs[j], a_tasks) \
                      for j in range(0, task_num + 1)] \
                     for i in range(0, agent_num)]

    move_vals = [[cal_MV(cur_con[i], task_cons[i][j], assigned[i], a_costs[i], algo) \
                  if (cur_cost + a_costs[i] <= budget) else 0 for j in range(0, task_num + 1)]
                 for i in range(0, agent_num)]

    iteration = 0
    re_assign = 0
    is_continue = True

    while is_continue:
        if time.time() - start_time > time_bound:
            break
        
        iteration += 1
        if move_vals == []:
            break
        a_index, t_index, max_v = choose_movement(move_vals, feasible_agents)

        if max_v <= 0:
            is_continue = False
        else:
            # perform move
            # remove agent from old task
            old_t_index = state[a_index]
            cs[old_t_index].remove(a_index)
            # add agent to new task
            state[a_index] = t_index
            cs[t_index].append(a_index)

            # recording task that are affected
            affected_t_indexes = []
            affected_t_indexes.append(t_index)
            if old_t_index != task_num:  # agent has been assigned to a task before
                affected_t_indexes.append(old_t_index)
                re_assign += 1
                if t_index == task_num:  # agent goes back to dummy
                    cur_cost -= a_costs[a_index]
                    affected_t_indexes.remove(t_index)
                    unassig_agents.append(a_index)
                    assign_agents.remove(a_index)
                    assigned[a_index] = False
            else:  # and old_t_index == task_num, agent move from dummy to a task
                cur_cost += a_costs[a_index]
                assign_agents.append(a_index)
                unassig_agents.remove(a_index)
                assigned[a_index] = True

            unfeasible_agents = []  # [i for i in unassig_agents if cur_cost + a_costs[i] > budget]
            fea_agents = []
            for i in unassig_agents:
                if cur_cost + a_costs[i] <= budget:
                    fea_agents.append(i)
                else:
                    unfeasible_agents.append(i)

            for j in range(0, task_num):
                for i in unfeasible_agents:
                    move_vals[i][j] = 0

            feasible_agents = assign_agents + fea_agents
            # contribution of agents to tasks
            for j in affected_t_indexes:
                for i in t_agents[j]:
                    if i in feasible_agents:
                        cur_con[i] = agent_con(competency_lists, t_caps_lists, i, state[i], cs[state[i]], a_tasks)
                        task_cons[i][j] = agent_con(competency_lists, t_caps_lists, i, j, cs[j], a_tasks)
                        move_vals[i][j] = cal_MV(cur_con[i], task_cons[i][j], assigned[i], a_costs[i], algo)
                        # notice that agents in affected_a_indexes has already been assigned, so don't need to check feasibility
            affected_a_indexes = list(chain.from_iterable([cs[j] for j in affected_t_indexes]))

            for i in affected_a_indexes:
                move_vals[i] = [cal_MV(cur_con[i], task_cons[i][j], assigned[i], a_costs[i], algo) for j in
                                range(0, task_num + 1)]
    sys_u = sys_reward(cs, competency_lists, t_caps_lists)

    return sys_u, cs, cur_cost, iteration




In [5]:
# def choose_swap(swap_vals, assign_agents, unassig_agents):
#     max_vals = [max(sv) for sv in swap_vals]
#     av_index, max_v = max(enumerate(max_vals), key=operator.itemgetter(1)) # av_index the max agent's index in assign_agents
#     a_index = assign_agents[av_index]
#     uv_index = swap_vals[av_index].index(max_v) #uv_index the swaping agent's index in unassign_agents
#     u_index = unassig_agents[uv_index]
#     return av_index, uv_index, a_index, u_index, max_v


# def swap_deass(t_caps_lists, competency_lists, a_tasks, budget, a_costs, cs, state, assign_agents, unassig_agents, cur_cost):
#     task_num = len(t_caps_lists)
#     # cs, state, assign_agents = deassign(t_caps_lists, competency_lists, a_tasks, a_costs, cs, state, assign_agents, assign_agents, cur_cost)
#     ass_efficiency = [agent_con(competency_lists, t_caps_lists, a, state[a], cs[state[a]], a_tasks)/a_costs[a] for a in assign_agents]
#     zero_eff = np.where(np.array(ass_efficiency) == 0)  # the index of the agent in the assigned_agent set
#     zero_eff_agents = zero_eff[0]  # store the index of the non-contribution agents
#     de_assign_agents = []
#     for k in zero_eff_agents:
#         de_assign_agents.append(assign_agents[k])
#     if len(zero_eff_agents) != 0:  # de-assign non-contribution agents
#         for k in de_assign_agents:
#             cur_cost -= a_costs[k]
#             org_task = state[k]
#             cs[org_task].remove(k)
#             cs[task_num].append(k)
#             state[k] = task_num
#             assign_agents.remove(k)
#             unassig_agents.append(k)

#     other_members = [list(filter(lambda x: x != a, cs[state[a]])) for a in assign_agents]
#     swap_vals = [[task_reward(other_members[ind] + [u], competency_lists, t_caps_lists[state[a]]) -
#                   task_reward(cs[state[a]], competency_lists, t_caps_lists[state[a]]) \
#                       if (cur_cost - a_costs[a] + a_costs[u] <= budget) and (state[a] in a_tasks[u]) else 0
#                   for u in unassig_agents]
#                  for ind, a in enumerate(assign_agents)]
    
#     is_continue = True
#     start_time = time.time()
#     while is_continue:
#         if time.time() - start_time > 300:
#             break
#         av_index, uv_index, a_index, u_index, max_v = choose_swap(swap_vals, assign_agents, unassig_agents)
#         if max_v <= 0:
#             is_continue = False
#         else:
#             # perform swap
#             # remove agent a from task to dummy
#             cur_cost -= a_costs[a_index]
#             cur_cost += a_costs[u_index]
#             t_index = state[a_index]
#             state[a_index] = task_num
#             cs[t_index].remove(a_index)
#             cs[task_num].append(a_index)

#             # add agent u to task
#             state[u_index] = t_index
#             cs[t_index].append(u_index)
#             cs[task_num].remove(u_index)

#             # swap a and u in assign_agents and unassig_agents
#             assign_agents[av_index] = u_index
#             unassig_agents[uv_index] = a_index

#             other_members = [list(filter(lambda x: x != a, cs[state[a]])) for a in assign_agents]
#             swap_vals = [[task_reward(other_members[ind] + [u], competency_lists, t_caps_lists[state[a]]) -
#                           task_reward(cs[state[a]], competency_lists, t_caps_lists[state[a]]) \
#                               if (cur_cost - a_costs[a] + a_costs[u] <= budget) and (state[a] in a_tasks[u]) else 0
#                           for u in unassig_agents]
#                          for ind, a in enumerate(assign_agents)]

#     sys_u = sys_reward(cs, competency_lists, t_caps_lists)
#     return sys_u, cs, cur_cost




In [6]:
########## general GA ##############
# def random_good_initial(state):
#     new_state = state[:]
#     select_ind = np.random.choice(list(range(0, len(state))), size=2, replace=False)
#     new_state[select_ind[0]] = state[select_ind[1]]
#     new_state[select_ind[1]] = state[select_ind[0]]
            
#     return new_state

# fitness
def fitness_GA(CS, competency_lists, t_caps_lists):
    ini_task_u = [[] for j in range(0, len(CS)-1)]
    for j in range(0, len(CS)-1):
        members = CS[j]
        if not members:
            ini_task_u[j] = 0
        else:
            ini_task_u[j] = task_reward(members, competency_lists, t_caps_lists[j])
    global_u = sum(ini_task_u)
    return global_u #, ini_task_u


def state_u_c(alloc, competency_lists, t_caps_lists, a_costs, B):
    task_num = len(t_caps_lists)
    CS = [[] for j in range(0, task_num+1)]
    sys_cost = 0
    for i in range(0,agent_num):
        CS[alloc[i]].append(i)
        if alloc[i] != task_num:
            sys_cost += a_costs[i]        
    sys_u = fitness_GA(CS, competency_lists, t_caps_lists) if sys_cost <= B else 0    
    return sys_u


def gen_population(population_size, competency_lists, t_caps_lists, a_costs, a_tasks, B, ini_popu=[]):
    task_num = len(t_caps_lists)
    population = {}
    for k in range(0, population_size - len(ini_popu)):
        state = random_in_budget(a_tasks, a_costs, task_num, B)
        sys_u = state_u_c(state, competency_lists, t_caps_lists, a_costs, B)
        population['inidi' + str(k)] = state
        population['fitness' + str(k)] = sys_u
    for j in range(0, len(ini_popu)):
        population['inidi' + str(population_size - j - 1)] = ini_popu[j]['inidi'][:]
        population['fitness' + str(population_size - j - 1)] = ini_popu[j]['fitness']
    return population


def GA_func(competency_lists, t_caps_lists, a_costs, a_tasks, B, population, pc, pm, m_points, iter_bound, time_bound):

    agent_num = len(competency_lists)
    task_num = len(t_caps_lists)
    population_size = int(len(population) / 2)

    start_time = time.time()
    iter = 1
    record_u = []
    while True:  # iter <= iter_bound:  #
        if time.time() - start_time > time_bound:
            break
        if iter > iter_bound:
            if record_u[-1]-record_u[-iter_bound]==0:
                break
        ############ selection ###################
        current_fit = []
        for k in list(range(0, population_size)):
            current_fit.append(population['fitness' + str(k)])
        weight = [current_fit[i] / sum(current_fit) for i in range(0, population_size)]

        ############ generate new poputation ############
        new_population = {}
        new_population_size = 0
        while new_population_size < population_size:
            select_parents = np.random.choice(list(range(0, population_size)), size=2, replace=False, p=weight)
            parent0 = population['inidi' + str(select_parents[0])][:]
            parent1 = population['inidi' + str(select_parents[1])][:]
            ########### start crossover ################
            R_rand = np.random.random(1)
            if R_rand <= pc:
                crossover_point = np.random.choice(list(range(0, agent_num)))
                new_population['inidi' + str(new_population_size)] = [parent0[i] if i <= crossover_point else
                                                                      parent1[i] for i in range(0, agent_num)]
                new_population['inidi' + str(new_population_size + 1)] = [parent1[i] if i <= crossover_point else
                                                                          parent0[i] for i in range(0, agent_num)]
            else:
                new_population['inidi' + str(new_population_size)] = parent0[:]
                new_population['inidi' + str(new_population_size + 1)] = parent1[:]
            ########### end crossover ################

            ########### start mutation ################
            M_rand = np.random.random(2)
            index_k = 0
            for k in list(range(0, 2)):
                if M_rand[index_k] <= pm:
                    mutation_points = list(
                        np.random.choice(list(range(0, agent_num)), m_points, replace=False)) if m_points != 1 else [
                        np.random.choice(list(range(0, agent_num)))]  # randomly generate the mutation points.
                    for m in range(0, m_points):
                        org_gene = new_population['inidi' + str(new_population_size + k)][mutation_points[m]]
                        other_genes = list(filter(lambda x: x != org_gene, a_tasks[mutation_points[m]]))
                        new_population['inidi' + str(new_population_size + k)][mutation_points[m]] = np.random.choice(
                            other_genes)
                new_population['fitness' + str(new_population_size + k)] = state_u_c(new_population['inidi' + str(new_population_size + k)], competency_lists, t_caps_lists, a_costs, B)
                index_k += 1
                ########### end mutation ################
            new_population_size += 2

        ########## strat update population ################
        new_generation_fit = []
        for k in list(range(0, population_size)):
            new_generation_fit.append(new_population['fitness' + str(k)])

        compare_fitness = current_fit + new_generation_fit
        rank_com_ind = [index for index, value in
                        sorted(list(enumerate(compare_fitness)), key=lambda x: x[1], reverse=True)]
        for k in list(range(0, population_size)):
            if rank_com_ind[k] < population_size:
                population['inidi' + str(k)] = population['inidi' + str(rank_com_ind[k])][:]
                population['fitness' + str(k)] = population['fitness' + str(rank_com_ind[k])]
            else:
                population['inidi' + str(k)] = new_population['inidi' + str(rank_com_ind[k] - population_size)][:]
                population['fitness' + str(k)] = new_population['fitness' + str(rank_com_ind[k] - population_size)]
        ########## end update population ################
        iter += 1
        record_u.append(population['fitness' + str(0)])
        ############### results ######################
    GA_solution = population['inidi' + str(0)]
    GA_u = population['fitness' + str(0)]
    GA_cost = 0
    for i in range(0, agent_num):
        if GA_solution[i] != task_num:
            GA_cost += a_costs[i]
    return GA_u, GA_solution, GA_cost, iter-1, record_u


In [7]:
class cartesian(object):
    def __init__(self):
        self._data_list=[]

    def add_data(self,data=[]): # add in data to generate the list of Cartesian
        self._data_list.append(data)

    def build(self): # calculate Cartesian
        all_combination = []
        for item in itertools.product(*self._data_list):
            all_combination.append(list(item))
        return all_combination


def combination(combine_list):
    # combine_list: the domain of agents
    car = cartesian()
    for i in range(0, len(combine_list)):
        car.add_data(list(combine_list[i]))
    comb = car.build()
    return comb

def exhaustive_search(t_caps_lists, competency_lists, a_tasks, B, a_costs):
    task_num = len(t_caps_lists)
    domains = np.array(a_tasks)
    all_comb = combination(domains)
    all_results = []
    all_costs = []
    for i in range(0, len(all_comb)):
        CS = [[] for j in range(0, task_num + 1)]
        total_cost = 0
        ini_alloc = list(all_comb[i])  # agents' selections, an allocation solution
        selected_tasks = list(set(ini_alloc))
        dummy_index = np.where(np.array(selected_tasks) == task_num)
        if len(dummy_index[0]) != 0:
            del selected_tasks[int(dummy_index[0])]
        for j in selected_tasks:
            members = np.where(np.array(ini_alloc) == j)
            CS[j] = list(members[0])
            task_cost = sum(a_costs[k] for k in CS[j])
            total_cost += task_cost
        if total_cost > B:
            reached_u = 0
        else:
            reached = sys_reward(CS, competency_lists, t_caps_lists)
            reached_u = reached[0]

        all_results.append(reached_u)
        all_costs.append(total_cost)
    optimal_u = max(all_results)
    index = np.where(np.array(all_results) == optimal_u)
    compare_cost = []
    for x in list(index[0]):
        compare_cost.append(all_costs[x])

    opt_cost = min(compare_cost)
    cost_min_ind = np.where(np.array(compare_cost) == opt_cost)
    index_com = cost_min_ind[0][0]
    opt_index = index[0][index_com]
    opt_structure = all_comb[opt_index]

    OPT_CS = [[] for j in range(0, task_num + 1)]
    allocation_str = list(set(opt_structure))
    for j in allocation_str:
        OPT_members = np.where(np.array(opt_structure) == j)
        OPT_CS[j] = list(OPT_members[0])

    return optimal_u, OPT_CS, opt_cost

In [8]:
def append_record(record, filename, typ):
    with open(filename, 'a') as f:
        if typ != '':
            json.dump(record, f, default = typ)
        else:
            json.dump(record, f)
        f.write('\n')
        # f.write(os.linesep)
        f.close()

In [9]:
run_num = 1000
sd = 0.3
T=[5, 10, 15, 20]
capNum = 20
capabilities = list(range(0, capNum))
max_capNum_task = 20
max_capNum_agent = 20
max_capVal = 10
c_min = 1  # minimum cost
c_max = 20  # maximum cost
time_bound = 1000  # time bound for GA
iter_bound = 100 # math.inf  # the generation of offspring
population_size = 100  # the initial population size
pc = 0.9  # crossover rate
pm = 0.5  # mutation rate
m_points = 1  # number of mutation points
ex_identifier = 0


# compare algorithms
for run in range(0, run_num):
    print('run:', run)

    for task_num in T:
        ex_identifier += 1
        
        agent_num = 3*task_num                    
        B = 5 * task_num   
        # generating tasks (representing as capabilities it needs)
        t_caps_lists = gen_tasks(task_num, max_capNum_task, capabilities)
        # randomly generate a list of tasks feasible for an agent (constraints); an agent can work on [1,task_num/2] tasks
        a_tasks = [list(np.random.choice(range(0, task_num), np.random.randint(1, task_num/2+1), replace=False))
                   for i in range(0, agent_num)]

        # get also the list of agents for each task
        t_agents = [[i for i in range(0, agent_num) if j in a_tasks[i]] for j in range(0, task_num)]

        # generating agents, including their capabilities, competency values, and costs
        a_caps_lists, competency_lists, a_costs = gen_agents(a_tasks, t_caps_lists, max_capNum_agent, capabilities,
                                                    max_capVal, c_min, c_max, sd)

        for i in range(0, agent_num):
            a_tasks[i].append(task_num)
        t_agents.append(list(range(0, agent_num)))

        
        result = {"ex_identifier": ex_identifier, "task_num": task_num, "agent_num": agent_num}


        ##  CF
        start = time.time()
        [CF_u, CF_CS, CF_cost, CF_iter] = TA_func(t_caps_lists, competency_lists, a_tasks, t_agents, B, a_costs, [],
                                                  0, "CF", time_bound)
        end = time.time()
        result['CF_u'] = CF_u
        result['CF_c'] = int(CF_cost)
        result['CF_t'] = end - start
        result['CF_iter'] = CF_iter
        print("task number: ", task_num, 'CF utility:', CF_u,
              "CF cost:", CF_cost, "CF time:", end - start, 'CF iteration:', CF_iter)
        print()

        # GA
        start = time.time()
        population = gen_population(population_size, competency_lists, t_caps_lists, a_costs, a_tasks, B, [])
        [GA_u, GA_solution, GA_cost, GA_iteration, GA_record] = GA_func(competency_lists, t_caps_lists, a_costs, a_tasks, B,
                                                             population, pc, pm, m_points, iter_bound, time_bound)
        end = time.time()
        result['GA_u'] = GA_u
        result['GA_c'] = GA_cost
        result['GA_t'] = end - start
        result['GA_iter'] = GA_iteration
        print("task number: ", task_num, 'GA utility:', GA_u,
              "GA cost:", GA_cost, "GA time:", end - start, 'GA iteration:', GA_iteration)
        print()



        files = {'discuss_small': [result, '']}
        for filename in list(files.keys()):
            append_record(files[filename][0], filename, typ=files[filename][1])








run: 0
task number:  5 CF utility: 160 CF cost: 24 CF time: 0.0019941329956054688 CF iteration: 6

task number:  5 GA utility: 148 GA cost: 25 GA time: 1.0491900444030762 GA iteration: 105

task number:  10 CF utility: 461 CF cost: 49 CF time: 0.006951093673706055 CF iteration: 18

task number:  10 GA utility: 460 GA cost: 50 GA time: 1.741020679473877 GA iteration: 120

task number:  15 CF utility: 589 CF cost: 73 CF time: 0.010940313339233398 CF iteration: 24

task number:  15 GA utility: 569 GA cost: 75 GA time: 1.9650523662567139 GA iteration: 123

task number:  20 CF utility: 885 CF cost: 100 CF time: 0.017982006072998047 CF iteration: 28

task number:  20 GA utility: 770 GA cost: 100 GA time: 2.5248920917510986 GA iteration: 135

run: 1
task number:  5 CF utility: 144 CF cost: 21 CF time: 0.0009965896606445312 CF iteration: 9

task number:  5 GA utility: 139 GA cost: 24 GA time: 1.0771510601043701 GA iteration: 103

task number:  10 CF utility: 454 CF cost: 48 CF time: 0.00398898

task number:  5 GA utility: 240 GA cost: 25 GA time: 1.017448902130127 GA iteration: 105

task number:  10 CF utility: 376 CF cost: 48 CF time: 0.004989147186279297 CF iteration: 16

task number:  10 GA utility: 403 GA cost: 50 GA time: 1.5509660243988037 GA iteration: 121

task number:  15 CF utility: 689 CF cost: 74 CF time: 0.013957023620605469 CF iteration: 28

task number:  15 GA utility: 541 GA cost: 75 GA time: 2.0466039180755615 GA iteration: 122

task number:  20 CF utility: 839 CF cost: 99 CF time: 0.012964963912963867 CF iteration: 26

task number:  20 GA utility: 775 GA cost: 100 GA time: 2.240997791290283 GA iteration: 120

run: 12
task number:  5 CF utility: 124 CF cost: 23 CF time: 0.000997781753540039 CF iteration: 9

task number:  5 GA utility: 130 GA cost: 25 GA time: 0.9675705432891846 GA iteration: 104

task number:  10 CF utility: 341 CF cost: 47 CF time: 0.0060079097747802734 CF iteration: 16

task number:  10 GA utility: 304 GA cost: 50 GA time: 1.420181274414062

task number:  10 GA utility: 473 GA cost: 50 GA time: 1.7499680519104004 GA iteration: 119

task number:  15 CF utility: 774 CF cost: 75 CF time: 0.010972023010253906 CF iteration: 23

task number:  15 GA utility: 628 GA cost: 75 GA time: 1.9608402252197266 GA iteration: 117

task number:  20 CF utility: 1022 CF cost: 99 CF time: 0.020944595336914062 CF iteration: 37

task number:  20 GA utility: 891 GA cost: 100 GA time: 2.671971559524536 GA iteration: 134

run: 23
task number:  5 CF utility: 141 CF cost: 22 CF time: 0.0010025501251220703 CF iteration: 6

task number:  5 GA utility: 141 GA cost: 22 GA time: 1.0732061862945557 GA iteration: 102

task number:  10 CF utility: 375 CF cost: 50 CF time: 0.005984783172607422 CF iteration: 14

task number:  10 GA utility: 333 GA cost: 50 GA time: 1.6185169219970703 GA iteration: 113

task number:  15 CF utility: 732 CF cost: 75 CF time: 0.008975505828857422 CF iteration: 22

task number:  15 GA utility: 644 GA cost: 75 GA time: 1.994285583496

task number:  15 GA utility: 472 GA cost: 75 GA time: 1.678696870803833 GA iteration: 115

task number:  20 CF utility: 859 CF cost: 99 CF time: 0.015956878662109375 CF iteration: 28

task number:  20 GA utility: 802 GA cost: 100 GA time: 2.9433014392852783 GA iteration: 165

run: 34
task number:  5 CF utility: 143 CF cost: 20 CF time: 0.000997304916381836 CF iteration: 8

task number:  5 GA utility: 148 GA cost: 25 GA time: 1.075141191482544 GA iteration: 103

task number:  10 CF utility: 495 CF cost: 50 CF time: 0.003989458084106445 CF iteration: 14

task number:  10 GA utility: 467 GA cost: 50 GA time: 1.442295789718628 GA iteration: 117

task number:  15 CF utility: 584 CF cost: 75 CF time: 0.012965202331542969 CF iteration: 30

task number:  15 GA utility: 468 GA cost: 75 GA time: 1.6873209476470947 GA iteration: 115

task number:  20 CF utility: 836 CF cost: 100 CF time: 0.018945932388305664 CF iteration: 32

task number:  20 GA utility: 730 GA cost: 100 GA time: 2.58890509605407

task number:  20 GA utility: 805 GA cost: 100 GA time: 3.181344747543335 GA iteration: 156

run: 45
task number:  5 CF utility: 126 CF cost: 24 CF time: 0.000997304916381836 CF iteration: 7

task number:  5 GA utility: 155 GA cost: 25 GA time: 0.9504704475402832 GA iteration: 102

task number:  10 CF utility: 433 CF cost: 50 CF time: 0.003992557525634766 CF iteration: 14

task number:  10 GA utility: 424 GA cost: 50 GA time: 1.746549129486084 GA iteration: 127

task number:  15 CF utility: 582 CF cost: 75 CF time: 0.00800943374633789 CF iteration: 19

task number:  15 GA utility: 559 GA cost: 73 GA time: 1.9518218040466309 GA iteration: 126

task number:  20 CF utility: 889 CF cost: 99 CF time: 0.021944046020507812 CF iteration: 36

task number:  20 GA utility: 718 GA cost: 100 GA time: 2.846029043197632 GA iteration: 154

run: 46
task number:  5 CF utility: 209 CF cost: 25 CF time: 0.0009982585906982422 CF iteration: 5

task number:  5 GA utility: 209 GA cost: 25 GA time: 1.0572328567

...

task number:  20 GA utility: 636 GA cost: 100 GA time: 1.7853622436523438 GA iteration: 113

run: 948
task number:  5 CF utility: 149 CF cost: 25 CF time: 0.0009722709655761719 CF iteration: 6

task number:  5 GA utility: 149 GA cost: 24 GA time: 0.9726912975311279 GA iteration: 100

task number:  10 CF utility: 345 CF cost: 49 CF time: 0.003990650177001953 CF iteration: 14

task number:  10 GA utility: 325 GA cost: 50 GA time: 1.5450057983398438 GA iteration: 118

task number:  15 CF utility: 603 CF cost: 74 CF time: 0.010970354080200195 CF iteration: 23

task number:  15 GA utility: 508 GA cost: 75 GA time: 1.728522539138794 GA iteration: 118

task number:  20 CF utility: 869 CF cost: 100 CF time: 0.0219419002532959 CF iteration: 32

task number:  20 GA utility: 723 GA cost: 100 GA time: 2.3209128379821777 GA iteration: 132

run: 949
task number:  5 CF utility: 184 CF cost: 23 CF time: 0.0009980201721191406 CF iteration: 7

task number:  5 GA utility: 185 GA cost: 24 GA time: 1.07723

task number:  5 GA utility: 159 GA cost: 25 GA time: 0.9540812969207764 GA iteration: 102

task number:  10 CF utility: 491 CF cost: 49 CF time: 0.006014585494995117 CF iteration: 16

task number:  10 GA utility: 450 GA cost: 49 GA time: 1.5749297142028809 GA iteration: 112

task number:  15 CF utility: 825 CF cost: 72 CF time: 0.014929533004760742 CF iteration: 28

task number:  15 GA utility: 710 GA cost: 75 GA time: 2.1674981117248535 GA iteration: 129

task number:  20 CF utility: 859 CF cost: 97 CF time: 0.023937225341796875 CF iteration: 32

task number:  20 GA utility: 641 GA cost: 100 GA time: 2.142580986022949 GA iteration: 129

run: 960
task number:  5 CF utility: 118 CF cost: 24 CF time: 0.000997781753540039 CF iteration: 8

task number:  5 GA utility: 123 GA cost: 25 GA time: 0.9844019412994385 GA iteration: 103

task number:  10 CF utility: 379 CF cost: 45 CF time: 0.00498652458190918 CF iteration: 15

task number:  10 GA utility: 369 GA cost: 49 GA time: 1.586794376373291

task number:  10 GA utility: 237 GA cost: 50 GA time: 1.250765323638916 GA iteration: 106

task number:  15 CF utility: 684 CF cost: 73 CF time: 0.008975744247436523 CF iteration: 18

task number:  15 GA utility: 635 GA cost: 75 GA time: 1.8212733268737793 GA iteration: 116

task number:  20 CF utility: 784 CF cost: 100 CF time: 0.014960289001464844 CF iteration: 25

task number:  20 GA utility: 685 GA cost: 100 GA time: 2.6030099391937256 GA iteration: 157

run: 971
task number:  5 CF utility: 148 CF cost: 23 CF time: 0.0009965896606445312 CF iteration: 5

task number:  5 GA utility: 153 GA cost: 25 GA time: 0.9739973545074463 GA iteration: 103

task number:  10 CF utility: 272 CF cost: 48 CF time: 0.003988981246948242 CF iteration: 18

task number:  10 GA utility: 245 GA cost: 50 GA time: 1.3703784942626953 GA iteration: 116

task number:  15 CF utility: 626 CF cost: 74 CF time: 0.011937856674194336 CF iteration: 24

task number:  15 GA utility: 558 GA cost: 75 GA time: 1.82144236564

task number:  10 GA utility: 421 GA cost: 50 GA time: 1.5400078296661377 GA iteration: 121

task number:  15 CF utility: 661 CF cost: 73 CF time: 0.0069811344146728516 CF iteration: 19

task number:  15 GA utility: 613 GA cost: 75 GA time: 1.7971973419189453 GA iteration: 119

task number:  20 CF utility: 1055 CF cost: 99 CF time: 0.021971702575683594 CF iteration: 32

task number:  20 GA utility: 842 GA cost: 100 GA time: 2.56436824798584 GA iteration: 145

run: 982
task number:  5 CF utility: 86 CF cost: 23 CF time: 0.0010271072387695312 CF iteration: 7

task number:  5 GA utility: 95 GA cost: 25 GA time: 1.0006301403045654 GA iteration: 104

task number:  10 CF utility: 423 CF cost: 50 CF time: 0.0029916763305664062 CF iteration: 13

task number:  10 GA utility: 426 GA cost: 50 GA time: 1.4064390659332275 GA iteration: 114

task number:  15 CF utility: 460 CF cost: 74 CF time: 0.0059850215911865234 CF iteration: 14

task number:  15 GA utility: 357 GA cost: 75 GA time: 1.60385298728

task number:  15 GA utility: 590 GA cost: 75 GA time: 2.467411518096924 GA iteration: 149

task number:  20 CF utility: 1003 CF cost: 100 CF time: 0.018952131271362305 CF iteration: 31

task number:  20 GA utility: 802 GA cost: 100 GA time: 3.2628862857818604 GA iteration: 180

run: 993
task number:  5 CF utility: 218 CF cost: 25 CF time: 0.000997781753540039 CF iteration: 6

task number:  5 GA utility: 226 GA cost: 25 GA time: 1.0252909660339355 GA iteration: 103

task number:  10 CF utility: 465 CF cost: 50 CF time: 0.0029952526092529297 CF iteration: 10

task number:  10 GA utility: 485 GA cost: 50 GA time: 1.5121867656707764 GA iteration: 111

task number:  15 CF utility: 667 CF cost: 74 CF time: 0.012935400009155273 CF iteration: 25

task number:  15 GA utility: 544 GA cost: 75 GA time: 1.982405662536621 GA iteration: 124

task number:  20 CF utility: 819 CF cost: 99 CF time: 0.02293562889099121 CF iteration: 34

task number:  20 GA utility: 688 GA cost: 100 GA time: 2.13607859611